In [1]:
from utils.load_env_vars import load_env
load_env()

-----------------------------
Environment variables loaded:
HF_HOME /pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/hfcache
HUGGINGFACE_HUB_CACHE /pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/hfcache
WANDB_PROJECT llava-changes
TOKENIZERS_PARALLELISM true
-----------------------------


# prompt

In [2]:
judge_prompt= """
Hey, can you judge this prediction for my visual question answering dataset? The Answers can be given as a whole sentence or simple yes/no or attribute like 'color' or 'shape' or 'size' etc.

If the answer matches the Groundtruth give a score of one, else give a score of zero. Treat yes/no as a binary classification proble equal to 1 for yes and 0 for no, or true and false respectively.

For example, if the prediction is 'red' and the groundtruth is 'red', you should give a score of 1. If the prediction is 'red' and the groundtruth is 'blue', you should give a score of 0.

return a json object with the following format:
{{
    "prediction": "red",
    "groundtruth": "red",
    "score": 1
}}

Prediction:
{}

GT:
{}

"""


In [3]:
pred = """
assistant
No.
"""
gt = "False"


In [4]:
from judge_utils import query_sglang_llama3

In [5]:
response = query_sglang_llama3(pred, gt)

print(response)

import json
import re
json_response = re.search(r'{(.|\r?\n)*}', response).group()
json_response = json.loads(json_response)
print(json_response)


AttributeError: 'NoneType' object has no attribute 'format'

In [ ]:
judge_prompt= """
Hey, can you judge this prediction for my visual question answering dataset? The Answers can be given as a whole sentence or simple yes/no or attribute like 'color' or 'shape' or 'size' etc.

If the answer matches the Groundtruth give a score of one, else give a score of zero. Treat yes/no as a binary classification proble equal to 1 for yes and 0 for no, or true and false respectively.

For example, if the prediction is 'red' and the groundtruth is 'red', you should give a score of 1. If the prediction is 'red' and the groundtruth is 'blue', you should give a score of 0. An empty answer should get a score of 0.

return a json object with the following format:
{{
    "prediction": "red",
    "groundtruth": "red",
    "score": 1
}}

Prediction:
{}

GT:
{}

"""
import openai
client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="EMPTY")

pred = """
assistant
No.
"""
gt = "False"

preds = [pred, pred, pred]
gts = [gt, gt, gt]
def query_sglang_llama3_batched(preds, gts):
    responses = client.chat.completions.create(
        model="default",
        messages=[
            {"role": "user", "content": judge_prompt.format(pred, gt)}
            for pred, gt in zip(preds, gts)
        ]
    )

    return [choice.message.content for choice in responses.choices]



responses = query_sglang_llama3_batched(preds, gts)


In [ ]:
for i in responses:
    print(i)

Here is the judgment:

{
    "prediction": "No",
    "groundtruth": "False",
    "score": 1
}

Explanation: The prediction "No" matches the groundtruth "False", which is equivalent to 0 in a binary classification problem. So, the score is 1.
